In [ ]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 13.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import contractions
from textblob import TextBlob
import tqdm

In [ ]:
# Downloading the necessary NLTK resources
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# Loading the dataset and performing basic checks
dataset_path_train = "train.csv"
dataset_path_test = "test.csv"

# Loading the train dataset
train_df = pd.read_csv(dataset_path_train, header=None, names=["stars", "review"], encoding="utf-8")
print("Train Dataset Info:")
train_df.info()
print("Train Dataset Head:")
print(train_df.head())
print("Train Dataset Missing Values:")
print(train_df.isnull().sum())

Train Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650000 entries, 0 to 649999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   stars   650000 non-null  int64 
 1   review  650000 non-null  object
dtypes: int64(1), object(1)
memory usage: 9.9+ MB
Train Dataset Head:
   stars                                             review
0      5  dr. goldberg offers everything i look for in a...
1      2  Unfortunately, the frustration of being Dr. Go...
2      4  Been going to Dr. Goldberg for over 10 years. ...
3      4  Got a letter in the mail last week that said D...
4      1  I don't know what Dr. Goldberg was like before...
Train Dataset Missing Values:
stars     0
review    0
dtype: int64


In [ ]:
# Loading the test dataset
test_df = pd.read_csv(dataset_path_test, header=None, names=["stars", "review"], encoding="utf-8")
print("Test Dataset Info:")
test_df.info()
print("Test Dataset Head:")
print(test_df.head())
print("Test Dataset Missing Values:")
print(test_df.isnull().sum())

Test Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   stars   50000 non-null  int64 
 1   review  50000 non-null  object
dtypes: int64(1), object(1)
memory usage: 781.4+ KB
Test Dataset Head:
   stars                                             review
0      1  I got 'new' tires from them and within two wee...
1      1  Don't waste your time.  We had two different p...
2      1  All I can say is the worst! We were the only 2...
3      1  I have been to this restaurant twice and was d...
4      1  Food was NOT GOOD at all! My husband & I ate h...
Test Dataset Missing Values:
stars     0
review    0
dtype: int64


In [ ]:
# Dropping 3-star reviews and converting ratings to binary sentiment labels
train_df = train_df[train_df["stars"] != 3]
train_df["sentiment"] = train_df["stars"].apply(lambda x: 1 if x >= 4 else 0)
train_df.drop(columns=["stars"], inplace=True)
print("Train Sentiment Distribution:")
print(train_df["sentiment"].value_counts())

test_df = test_df[test_df["stars"] != 3]
test_df["sentiment"] = test_df["stars"].apply(lambda x: 1 if x >= 4 else 0)
test_df.drop(columns=["stars"], inplace=True)
print("Test Sentiment Distribution:")
print(test_df["sentiment"].value_counts())

Train Sentiment Distribution:
sentiment
1    260000
0    260000
Name: count, dtype: int64
Test Sentiment Distribution:
sentiment
0    20000
1    20000
Name: count, dtype: int64


<ipython-input-9-8eb810114d77>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["sentiment"] = test_df["stars"].apply(lambda x: 1 if x >= 4 else 0)
<ipython-input-9-8eb810114d77>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(columns=["stars"], inplace=True)


In [ ]:
# Function to expand contractions in text (e.g., "don't" -> "do not")
def expand_contractions(text):
    return contractions.fix(text)

# Function to reduce repeated characters (e.g., "sooo goood" -> "soo good")
def reduce_repeated_chars(text):
    return re.sub(r'(.)\1{2,}', r'\1\1', text)

# Function to remove HTML tags from text
def remove_tag(text):
    return re.sub('<[^<]+?>', '', text)

# Function to remove URLs from text
def remove_urls(text):
    rgx = re.compile(r"((http://)[^ ]*|(https://)[^ ]*|(www\.)[^ ]*)")
    return rgx.sub(' ', text)

# Function to remove alphanumeric words (e.g., "abc123")
def remove_alpha_numeric(text):
    rgx = re.compile(r'\d+\w+|\w+\d+|\d+')
    return rgx.sub(' ', text)

# Function to strip punctuation from text
def strip_punctuation(text):
    text = [char for char in text if char not in string.punctuation]
    return ''.join(text)

In [ ]:
# Function to preprocess review text
def preprocess_text(text):

    text = text.lower()

    text = expand_contractions(text)

    text = remove_tag(text)

    text = remove_urls(text)

    text = reduce_repeated_chars(text)

    text = remove_alpha_numeric(text)

    text = strip_punctuation(text)

    tokens = word_tokenize(text)

    # Removing stopwords
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]

    # Applying lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return " ".join(tokens)

In [ ]:
print(f"Null values in train dataset:\n{train_df.isnull().sum()}")
print(f"Null values in test dataset:\n{test_df.isnull().sum()}")

Null values in train dataset:
review       0
sentiment    0
dtype: int64
Null values in test dataset:
review       0
sentiment    0
dtype: int64


In [ ]:
from tqdm.notebook import tqdm

tqdm.pandas()

# Applying the preprocessing function to the train and test sets
print("Preprocessing Train Dataset:")
train_df["processed_review"] = train_df["review"].progress_apply(preprocess_text)

print("Preprocessing Test Dataset:")
test_df["processed_review"] = test_df["review"].progress_apply(preprocess_text)

Preprocessing Train Dataset:


  0%|          | 0/520000 [00:00<?, ?it/s]

Preprocessing Test Dataset:


  0%|          | 0/40000 [00:00<?, ?it/s]

In [ ]:
train_df[["review", "processed_review"]].head()

,review,processed_review
0,dr. goldberg offers everything i look for in a...,dr goldberg offer everything look general prac...
1,"Unfortunately, the frustration of being Dr. Go...",unfortunately frustration dr goldberg patient ...
2,Been going to Dr. Goldberg for over 10 years. ...,going dr goldberg year think one patient start...
3,Got a letter in the mail last week that said D...,got letter mail last week said dr goldberg mov...
4,I don't know what Dr. Goldberg was like before...,know dr goldberg like moving arizona let tell ...


In [ ]:
test_df[["review", "processed_review"]].head()

,review,processed_review
0,I got 'new' tires from them and within two wee...,got new tire within two week got flat took car...
1,Don't waste your time. We had two different p...,waste time two different people come house giv...
2,All I can say is the worst! We were the only 2...,say worst people place lunch place freezing lo...
3,I have been to this restaurant twice and was d...,restaurant twice disappointed time go back fir...
4,Food was NOT GOOD at all! My husband & I ate h...,food good husband ate couple week ago first ti...


In [ ]:
print(f"Null values in train dataset:\n{train_df.isnull().sum()}")
print(f"Null values in test dataset:\n{test_df.isnull().sum()}")

Null values in train dataset:
review              0
sentiment           0
processed_review    0
dtype: int64
Null values in test dataset:
review              0
sentiment           0
processed_review    0
dtype: int64


In [ ]:
blank_counts = (train_df.isna() | train_df.applymap(lambda x: isinstance(x, str) and x.strip() == '')).sum()
print(blank_counts)

<ipython-input-26-1b82d22e393f>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  blank_counts = (train_df.isna() | train_df.applymap(lambda x: isinstance(x, str) and x.strip() == '')).sum()


review               0
sentiment            0
processed_review    49
dtype: int64


In [ ]:
blank_counts = (test_df.isna() | test_df.applymap(lambda x: isinstance(x, str) and x.strip() == '')).sum()
print(blank_counts)

review              0
sentiment           0
processed_review    2
dtype: int64


<ipython-input-27-c8a73b58ec2b>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  blank_counts = (test_df.isna() | test_df.applymap(lambda x: isinstance(x, str) and x.strip() == '')).sum()


### **Note:**
### The train and test datasets had a few reviews which got processed and came out as a blank string value. Hence, we have dropped these rows and arrived at the final cleaned and preprocessed dataset.

In [ ]:
train_df_cleaned = train_df[~(train_df.isna() | train_df.applymap(lambda x: isinstance(x, str) and x.strip() == '')).any(axis=1)]
test_df_cleaned = test_df[~(test_df.isna() | test_df.applymap(lambda x: isinstance(x, str) and x.strip() == '')).any(axis=1)]

<ipython-input-29-b524eb440979>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_df_cleaned = train_df[~(train_df.isna() | train_df.applymap(lambda x: isinstance(x, str) and x.strip() == '')).any(axis=1)]
<ipython-input-29-b524eb440979>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_df_cleaned = test_df[~(test_df.isna() | test_df.applymap(lambda x: isinstance(x, str) and x.strip() == '')).any(axis=1)]


In [ ]:
blank_counts = (train_df_cleaned.isna() | train_df_cleaned.applymap(lambda x: isinstance(x, str) and x.strip() == '')).sum()
print(blank_counts)

<ipython-input-30-6cd7b9e6601b>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  blank_counts = (train_df_cleaned.isna() | train_df_cleaned.applymap(lambda x: isinstance(x, str) and x.strip() == '')).sum()


review              0
sentiment           0
processed_review    0
dtype: int64


In [ ]:
blank_counts = (test_df_cleaned.isna() | test_df_cleaned.applymap(lambda x: isinstance(x, str) and x.strip() == '')).sum()
print(blank_counts)

review              0
sentiment           0
processed_review    0
dtype: int64


<ipython-input-31-55b2e4c7f2b9>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  blank_counts = (test_df_cleaned.isna() | test_df_cleaned.applymap(lambda x: isinstance(x, str) and x.strip() == '')).sum()


In [ ]:
# Saving the processed datasets
train_df_cleaned[["sentiment", "processed_review"]].to_csv("processed_train.csv", index=False)
test_df_cleaned[["sentiment", "processed_review"]].to_csv("processed_test.csv", index=False)